In [1]:
import pandas as pd
from pynput.mouse import Listener, Controller, Button
import time
import threading

player = 'btest'

mouse = Controller()
data = []  # List to store rows of the DataFrame
recording = False  # Flag to start recording after the first click
start_time = None  # To record time relative to the first click

def on_click(x, y, button, pressed):
    global recording, start_time
    if pressed:
        if button == Button.right:
            # Stop recording and exit when right button is clicked
            print(f"Right click detected at {x}, {y}, stopping.")
            return False  # Returning False stops the listener
        if button == Button.left:
            if not recording:
                recording = True
                start_time = time.time()
            time_elapsed = time.time() - start_time
            data.append({'position': (x, y), 'time': time_elapsed, 'click': 1})
            # print(f"Mouse clicked at position: {x}, {y} at {time_elapsed} seconds")

def record_position():
    while True:
        if recording:
            pos = mouse.position
            time_elapsed = time.time() - start_time
            data.append({'position': pos, 'time': time_elapsed, 'click': 0})
            # print(f"Mouse position at {time_elapsed}: {pos}")
        time.sleep(0.1)

if __name__ == "__main__":
    position_thread = threading.Thread(target=record_position)
    position_thread.daemon = True  # Set daemon so it exits on main thread exit
    position_thread.start()

    # Listener for mouse clicks
    with Listener(on_click=on_click) as listener:
        listener.join()

    # Convert list of data to DataFrame
    df = pd.DataFrame(data)
    
    df = df[df['time']<=60]
    
    df['x'] = df['position'].apply(lambda x: x[0])
    df['y'] = df['position'].apply(lambda x: x[1])
    
    df = df[['x','y','time','click']]
    
    from datetime import datetime
    current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

    # File name with current timestamp
    file_name = f'{player}/data_{current_time}.csv'
    
    # Save the DataFrame to a CSV file
    df.to_csv(file_name, index=False)
    


Right click detected at 2274.5546875, 560.98046875, stopping.


In [2]:
#preprocessing
moves = []
df = df.loc[:df[df['click'] == 1].index[-1],:]
inds = df[df['click'] == 1].index
for i in range(len(inds)-1):
    dft = df.loc[inds[i]:inds[i+1]]
    dft = dft - dft.iloc[0,:]
    dft.drop('click', axis=1, inplace=True)
    moves.append(dft)

In [3]:
moves[1]

,x,y,time
11,0.0,0.0,0.000000
12,0.0,0.0,0.029192
13,-1.0,-1.0,0.130832
14,-14.0,-4.0,0.240394
15,-66.0,28.0,0.341406
16,-141.0,116.0,0.446651
17,-177.0,146.0,0.551740
18,-224.0,181.0,0.656976
19,-273.0,215.0,0.760757
20,-277.0,218.0,0.861117
